Title

In [1]:
from __future__ import division, print_function
%matplotlib inline

Setup Sympy

In [2]:
import sympy as sym
from sympy.physics.mechanics import *
from IPython.display import display
sym.init_printing(use_latex='mathjax')

In [3]:
def calcBfromQ(qvec):
    """
    dq/dt = 1/2 [B(q)] w
    """
    w = qvec[0]
    x = qvec[1]
    y = qvec[2]
    z = qvec[3]
    
    B = sym.zeros(4,3)
    B[0,0] = -x
    B[0,1] = -y
    B[0,2] = -z
    B[1,0] =  w
    B[1,1] = -z
    B[1,2] =  y
    B[2,0] =  z
    B[2,1] =  w
    B[2,2] = -x
    B[3,0] = -y
    B[3,1] =  x
    B[3,2] =  w

    return B

def skew(x):
    """
    Returns skew symetric matic of x
    where x is a vector of length 3
    """
    xSkew = sym.zeros(3,3)
    xSkew[0,1] = -x[2]
    xSkew[0,2] =  x[1]
    xSkew[1,0] =  x[2]
    xSkew[1,2] = -x[0]
    xSkew[2,0] = -x[1]
    xSkew[2,1] =  x[0]
    return xSkew    

def build_inertia_tensor(Ivec):
    Ixx,Iyy,Izz,Ixy,Ixz,Iyz = Ivec
    
    Imat = sym.zeros(3,3)
    Imat[0,0] = Ixx
    Imat[0,1] = Ixy
    Imat[0,2] = Ixz
    Imat[1,0] = Ixy
    Imat[1,1] = Iyy
    Imat[1,2] = Iyz
    Imat[2,0] = Ixz
    Imat[2,1] = Iyz
    Imat[2,2] = Izz
    
    return Imat

def quatConj(q):
    q_out = sym.zeros(4,1)
    q_out[0] = q[0]
    q_out[1] = -q[1]
    q_out[2] = -q[2]
    q_out[3] = -q[3]
    
    return q_out

def quatMult(a,b):
    
    w = b[0]*a[0]-b[1]*a[1]-b[2]*a[2]-b[3]*a[3]
    x = b[1]*a[0]+b[0]*a[1]+b[3]*a[2]-b[2]*a[3]
    y = b[2]*a[0]-b[3]*a[1]+b[0]*a[2]+b[1]*a[3]
    z = b[3]*a[0]+b[2]*a[1]-b[1]*a[2]+b[0]*a[3]    
    
    q = sym.Matrix([w,x,y,z])
    
    return q

def quatRot(q_toBfromA, v_expA):
    
    v_expA = sym.Matrix([0,v_expA])
    
    v_expB = quatMult( quatConj(q_toBfromA), quatMult(v_expA, q_toBfromA) ) 
    
    v_out = sym.zeros(3,1)
    v_out[0] = v_expB[1]
    v_out[1] = v_expB[2]
    v_out[2] = v_expB[3]
    
    return v_out
    
    

In [4]:
# Define Variables
x, y, z = sym.symbols('x, y, z') # Pos of B wrt NED exp in B Frame
vx, vy, vz = sym.symbols('vx, vy, vz') # Velocity of B wrt NED exp in B frame
qw, qx, qy, qz = sym.symbols('q_w, q_x, q_y, q_z')
wx, wy, wz = sym.symbols('w_x, w_y, w_z')
Ixx, Iyy, Izz, Ixy, Ixz, Iyz = sym.symbols('I_xx, I_yy, I_zz, I_xy, I_xz, I_yz')
Mx, My, Mz = sym.symbols('M_x, M_y, M_z') # Body Frame moments
Fx, Fy, Fz = sym.symbols('F_x, F_y, F_z') # Body Frame forces
F1, F2, F3, F4 = sym.symbols('F1, F2, F3, F4') # Motor Forces
M1, M2, M3, M4 = sym.symbols('M1, M2, M3, M4') # Motor torques
wm1, wm2, wm3, wm4 = sym.symbols('w_m1, w_m2, w_m3, w_m4') # Motor speeds
kF, kM = sym.symbols('k_F, k_M') # Motor force and moment coeffecients

mass, grav, L = sym.symbols('m, g, L')


# Setup Vectors
rvec = sym.Matrix([x,y,z])       # R of B wrt L exp B
qvec = sym.Matrix([qw,qx,qy,qz]) # attitude quaterion of B wrt L
wvec = sym.Matrix([wx,wy,wz])    # AngVel of B wrt L exp B
Fvec = sym.Matrix([Fx,Fy,Fz])    # Body Frame forces
Mvec = sym.Matrix([Mx,My,Mz])    # Body Frame moments
vvec = sym.Matrix([vx,vy,vz])    # Vel of B wrt L exp V

# Build Inertia Tensor
Ixy = 0
Ixz = 0
Iyz = 0
Ivec = Ixx,Iyy,Izz,Ixy,Ixz,Iyz
inertiaTensor = build_inertia_tensor(Ivec)
inertiaTensorInv = inertiaTensor.inv()

Solve dynamics equations for state space form

In [7]:
useMotorFM = True

# Motor Thrust and Torque
motor1_F = kF*wm1**2
motor2_F = kF*wm2**2
motor3_F = kF*wm3**2
motor4_F = kF*wm4**2
motor1_M = kM*wm1**2
motor2_M = kM*wm2**2
motor3_M = kM*wm3**2
motor4_M = kM*wm4**2

wmvec = sym.Matrix([wm1, wm2, wm3, wm4])
wmDot = sym.Matrix([0,0,0,0])

# Sub sum of body forces
print('Sum of Forces')
display(Fvec)
if useMotorFM is True:
    Fvec = Fvec.subs(Fx, 0 )
    Fvec = Fvec.subs(Fy, 0 )
    Fvec = Fvec.subs(Fz, -(F1+F2+F3+F4) )
    print('Sum of Forces')
    display(Fvec)
    mprint(Fvec)

# Sub sum of body moments
print('Sum of Moments')
display(Mvec)
if useMotorFM is True:
    Mvec = Mvec.subs(Mx, L*(F4-F2) )
    Mvec = Mvec.subs(My, L*(F1-F3) )
    Mvec = Mvec.subs(Mz, M2+M4-M1-M3 )
    print('Sum of Moments')
    display(Mvec)
    mprint(Mvec)

#Sum of Forces solved for linear accel in body frame
gravvec = sym.Matrix([0,0,grav])
vDot = 1/mass*Fvec + quatRot(qvec, gravvec) - skew(wvec)*vvec
print('vDot (Expressed in B frame)')
display(vDot)

# Sum of Moments solved for angular accel in body frame
wDot = inertiaTensorInv*(-skew(wvec)*inertiaTensor*wvec + Mvec)
print('wDot (Expressed in B frame)')
display(wDot)

# Quaternion rotational kinematics equation (quaterion to B from L)
B = calcBfromQ(qvec)
qDot = 0.5*B*wvec
print('qDot')
display(qDot)

Sum of Forces


⎡        0         ⎤
⎢                  ⎥
⎢        0         ⎥
⎢                  ⎥
⎣-F₁ - F₂ - F₃ - F₄⎦

Sum of Forces


⎡        0         ⎤
⎢                  ⎥
⎢        0         ⎥
⎢                  ⎥
⎣-F₁ - F₂ - F₃ - F₄⎦

Matrix([
[                 0],
[                 0],
[-F1 - F2 - F3 - F4]])
Sum of Moments


...

⎡             -2⋅g⋅q_w⋅q_y + 2⋅g⋅qₓ⋅q_z + vy⋅w_z - vz⋅w_y              ⎤
⎢                                                                      ⎥
⎢              2⋅g⋅q_w⋅qₓ + 2⋅g⋅q_y⋅q_z - vx⋅w_z + vz⋅wₓ               ⎥
⎢                                                                      ⎥
⎢     2       2        2        2                    -F₁ - F₂ - F₃ - F₄⎥
⎢g⋅q_w  - g⋅qₓ  - g⋅q_y  + g⋅q_z  + vx⋅w_y - vy⋅wₓ + ──────────────────⎥
⎣                                                            m         ⎦

wDot (Expressed in B frame)


⎡ I_yy⋅w_y⋅w_z - I_zz⋅w_y⋅w_z + L⋅(-F₂ + F₄) ⎤
⎢ ────────────────────────────────────────── ⎥
⎢                    Iₓₓ                     ⎥
⎢                                            ⎥
⎢  -Iₓₓ⋅wₓ⋅w_z + I_zz⋅wₓ⋅w_z + L⋅(F₁ - F₃)   ⎥
⎢  ───────────────────────────────────────   ⎥
⎢                    I_yy                    ⎥
⎢                                            ⎥
⎢Iₓₓ⋅wₓ⋅w_y - I_yy⋅wₓ⋅w_y - M₁ + M₂ - M₃ + M₄⎥
⎢────────────────────────────────────────────⎥
⎣                    I_zz                    ⎦

qDot


⎡-0.5⋅qₓ⋅wₓ - 0.5⋅q_y⋅w_y - 0.5⋅q_z⋅w_z⎤
⎢                                      ⎥
⎢0.5⋅q_w⋅wₓ + 0.5⋅q_y⋅w_z - 0.5⋅q_z⋅w_y⎥
⎢                                      ⎥
⎢0.5⋅q_w⋅w_y - 0.5⋅qₓ⋅w_z + 0.5⋅q_z⋅wₓ ⎥
⎢                                      ⎥
⎣0.5⋅q_w⋅w_z + 0.5⋅qₓ⋅w_y - 0.5⋅q_y⋅wₓ ⎦

In [6]:
# State vector
state = sym.Matrix([rvec,qvec,vvec,wvec,wmvec])
print('State Vector')
display(state)
mprint(state)

# Matrix
A = sym.Matrix([vvec,qDot,vDot,wDot,wmDot])
print('\nState transition matrix')
display(A)
mprint(A)

State Vector


⎡ x ⎤
⎢   ⎥
⎢ y ⎥
⎢   ⎥
⎢ z ⎥
⎢   ⎥
⎢q_w⎥
⎢   ⎥
⎢qₓ ⎥
⎢   ⎥
⎢q_y⎥
⎢   ⎥
⎢q_z⎥
⎢   ⎥
⎢vx ⎥
⎢   ⎥
⎢vy ⎥
⎢   ⎥
⎢vz ⎥
⎢   ⎥
⎢wₓ ⎥
⎢   ⎥
⎢w_y⎥
⎢   ⎥
⎢w_z⎥
⎢   ⎥
⎢wₘ₁⎥
⎢   ⎥
⎢wₘ₂⎥
⎢   ⎥
⎢wₘ₃⎥
⎢   ⎥
⎣wₘ₄⎦

Matrix([
[   x],
[   y],
[   z],
[ q_w],
[ q_x],
[ q_y],
[ q_z],
[  vx],
[  vy],
[  vz],
[ w_x],
[ w_y],
[ w_z],
[w_m1],
[w_m2],
[w_m3],
[w_m4]])

State transition matrix


⎡                                  vx                                  ⎤
⎢                                                                      ⎥
⎢                                  vy                                  ⎥
⎢                                                                      ⎥
⎢                                  vz                                  ⎥
⎢                                                                      ⎥
⎢                -0.5⋅qₓ⋅wₓ - 0.5⋅q_y⋅w_y - 0.5⋅q_z⋅w_z                ⎥
⎢                                                                      ⎥
⎢                0.5⋅q_w⋅wₓ + 0.5⋅q_y⋅w_z - 0.5⋅q_z⋅w_y                ⎥
⎢                                                                      ⎥
⎢                0.5⋅q_w⋅w_y - 0.5⋅qₓ⋅w_z + 0.5⋅q_z⋅wₓ                 ⎥
⎢                                                                      ⎥
⎢                0.5⋅q_w⋅w_z + 0.5⋅qₓ⋅w_y - 0.5⋅q_y⋅wₓ                 ⎥
⎢                                                  

Matrix([
[                                                                                  vx],
[                                                                                  vy],
[                                                                                  vz],
[                                            -0.5*q_x*w_x - 0.5*q_y*w_y - 0.5*q_z*w_z],
[                                             0.5*q_w*w_x + 0.5*q_y*w_z - 0.5*q_z*w_y],
[                                             0.5*q_w*w_y - 0.5*q_x*w_z + 0.5*q_z*w_x],
[                                             0.5*q_w*w_z + 0.5*q_x*w_y - 0.5*q_y*w_x],
[                                        -2*g*q_w*q_y + 2*g*q_x*q_z + vy*w_z - vz*w_y],
[                                         2*g*q_w*q_x + 2*g*q_y*q_z - vx*w_z + vz*w_x],
[g*q_w**2 - g*q_x**2 - g*q_y**2 + g*q_z**2 + vx*w_y - vy*w_x + (-F1 - F2 - F3 - F4)/m],
[                                   (I_yy*w_y*w_z - I_zz*w_y*w_z + L*(-F2 + F4))/I_xx],
[                      

Additional parameters